Import packages:

In [487]:
from itertools import product
from sage.crypto.boolean_function import BooleanFunction

Define an even integer $n\geq 2$:

In [488]:
n = 20
n_half = n // 2

For a Boolean vector $a=(a_i)_{i\in[1,n]}\in\mathbb F_2^n$, define:
- $p=p(a)$ to be the number of odd integers $i$ with $(a_i,a_{i+1})=(0,0)$.
- $q=q(a)$ to be the number of odd integers $i$ with $(a_i,a_{i+1})=(1,1)$.
- $r=p(a)$ to be the number of odd integers $i$ with $(a_i,a_{i+1})\in\{(0,1),(1,0)\}$.

It then holds that $p+q+r=n/2$. Consider then the polynomial $P_a(z)=(-z^2+2z+1)^p\cdot(-z^2-2z+1)^q\cdot(z^2+1)^r$. We can write:
$$P_a(z)=\sum_{k=0}^n\mathsf D_{n,k,a}z^k,\qquad\mathsf D_{n,k,a}\in\mathbb Z.$$

In [489]:
D = {}

z = var("z")
P = -(z**2) + 2 * z + 1
Q = -(z**2) - 2 * z + 1
R = z**2 + 1

powers_P = [1]
powers_Q = [1]
powers_R = [1]

for r in range(1, n_half + 1):
    powers_P.append((powers_P[r - 1] * P).expand())
    powers_Q.append((powers_Q[r - 1] * Q).expand())
    powers_R.append((powers_R[r - 1] * R).expand())

for p in range(n_half + 1):
    for q in range(n_half - p + 1):
        r = n_half - p - q
        D[p, q] = (powers_P[p] * powers_Q[q] * powers_R[r]).list()

# Bounding the Walsh transform

For $a\in\mathbb F_2^n$, we define the following (modified) Walsh transform:
$$V(a)=1+\sum_{k=1}^n\mathsf D_{n,k,a+g_k},$$
where $g_k\in\mathbb F_2^n$ is the canonical basis vector having its $1$ at position $2k-1$ if $k\leq n/2$, and at position $2k-n$ if $k>n/2$.

The following finds a bound on $\max_{a\in\mathbb F_2^n}|V(a)|$:

In [490]:
def get_bound_V():
    upper_bounds = []
    lower_bounds = []
    for p in range(n_half + 1):
        for q in range(n_half - p + 1):
            tuples = []
            if p > 0:
                tuples.append((p - 1, q, p - 1, q))
            if q > 0:
                tuples.append((p, q - 1, p, q - 1))
            if p + q < n_half:
                tuples.append((p + 1, q, p, q + 1))
                tuples.append((p, q + 1, p + 1, q))
            upper_bound_pq = 0
            lower_bound_pq = 0
            for k in range(0, n_half + 1):
                upper_bound_pq += max(
                    D[p_1, q_1][k] + D[p_2, q_2][k + n_half]
                    for (p_1, q_1, p_2, q_2) in tuples
                )
                lower_bound_pq += min(
                    D[p_1, q_1][k] + D[p_2, q_2][k + n_half]
                    for (p_1, q_1, p_2, q_2) in tuples
                )
            upper_bounds.append(upper_bound_pq)
            lower_bounds.append(lower_bound_pq)
    U = max(upper_bounds)
    L = min(lower_bounds)
    return max(abs(U), abs(L))

In [491]:
get_bound_V()

392168